In [183]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import os
import re
import string
import pprint
import json
import time

In [156]:
data_dir = "C:\\data"
xls_path = os.path.join(data_dir,"company_list.xls")

In [157]:
df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]


In [158]:
company_object = df['회사명']
print(company_object)
company_list = list(company_object)

0            DSR
1          GS글로벌
2          HSD엔진
3          LG이노텍
4           LS산전
          ...   
2350       에이원알폼
2351       원바이오젠
2352     지앤이헬스케어
2353          코셋
2354    한국미라클피플사
Name: 회사명, Length: 2355, dtype: object


In [159]:
from collections import defaultdict
company_dict = defaultdict(list)

In [188]:
# html = urlopen("https://www.amazon.com/books-used-books-textbooks/b?ie=UTF8&node=283155")  
url ="https://biz.chosun.com"
html = urlopen(url)
bsObject = BeautifulSoup(html, "html.parser") 
bsObject_lxml = BeautifulSoup(html, 'lxml', from_encoding='utf-8')

# print(bsObject.prettify())

In [161]:
def preprocess_word(x) :
    m = x.replace('%','per')
    m = re.sub(r'(?:\b[0-9a-zA-Zㄱ-ㅎㅏ-ㅣ]\b|[?!\W]+)\s*', ' ', m).strip() 
    m = re.sub('per','%',m)
    m = " ".join(m.split())
    return m

In [162]:
def get_string_from_chosun(bsObject):
    text_list = []
    for i,meta in enumerate(bsObject.find_all("a")):
        text_list.append(meta.get_text())
    return text_list
#     print(i,bsObject.a['on_click'])

In [179]:
def add_company_info_from(s_list,company_dict,verbose=True):
    for s in s_list:
        
        processed_words = []
        for word in s.split(' '):
            new_word = preprocess_word(word)
            if new_word in ["속보","전체보기"]:
                continue
            elif new_word and new_word!='':
                processed_words.append(new_word)
        if processed_words and verbose:print('*'*20);print(processed_words)
        for word in processed_words:
            if word in company_list:
                company_dict[word].append(" ".join(processed_words))
        

In [180]:
company_dict.clear()
chosun_news = get_string_from_chosun(bsObject) 
add_company_info_from(chosun_news,company_dict)


********************
['임원', '월급', '30%', '반납 전직원', '무급휴가 제주항공', '생존', '위기']
********************
['일본', '현대重 대우조선', '합병', '반대', 'WTO', '제소']
********************
['갤럭시S20', '괴물', '카메라 에', '최초로', '들어간', '이', '기술', '뭐길래']
********************
['기생충만큼', '충격적', '외신', '갤럭시Z플립', '호평', '일색']
********************
['타다', '쏘카에서', '독립', '리스크', '분산 투자', '유치', '수월해질', '듯']
********************
['우한폐렴發', '충격', '우려', '홍남기', '경제수장', '6개월', '만에', '소집']
********************
['곽신애', '이미경', '수상소감', '뒷말에', '팀에서', '미리', '정해뒀다']
********************
['구지은', '멘토', '이미경', '사촌언니여서', '감사', '진심으로', '축하']
********************
['전', '세계', '도미노', '기생충', '열풍', '스페인서도', '박스오피스', '1위']
********************
['공식', '데뷔 한', '노태문', '사장', '삼성과', '애플의', '길은', '다르다']
********************
['알립니다', '사랑받는', '브랜드', '대상', '조선비즈가', '선정합니다']
********************
['1억화소 5G 폴더블 괴물스펙 으로', '닻', '올린', '노태문號']
********************
['1억화소 100배', '줌', '갤S20', '베일', '벗었다']
********************
['라임에', '달려든', '법무법인들', '광화 우리도', '소송전', '참전']
*

In [181]:
pprint.pprint(company_dict)
# print(list(company_list))

defaultdict(<class 'list'>,
            {'AK홀딩스': ['AK홀딩스 지난해 영업익 1310억원 전년比 51% 감소'],
             'CJ': ['CJ 제일제당 작년 매출 22조3525억원 역대 최대'],
             'KB금융': ['KB금융 위해 5500억 자금 조달'],
             '대상': ['알립니다 사랑받는 브랜드 대상 조선비즈가 선정합니다'],
             '동아에스티': ['동아에스티 작년 영업익 570억원 전년比 44 5% 증가'],
             '롯데관광개발': ['롯데관광개발 우한폐렴 피해 제주 소상공인에 10억 지원'],
             '삼성전자': ['삼성전자 아카데미 시상식서 갤럭시Z플립 공개', '삼성전자 아카데미 시상식서 갤럭시Z플립 공개'],
             '서울반도체': ['서울반도체 4분기 영업익 139억 올해 마이크로 LED 기대'],
             '씨젠': ['씨젠 질본에 우한폐렴 진단 시약 긴급 사용 승인 받아'],
             '아시아나항공': ['美中분쟁 日불매에 아시아나항공 작년 3683억 영업손실'],
             '오스템임플란트': ['오스템임플란트 AI 기반 디지털 치과용 전자차트 내놓는다'],
             '이마트': ['초저가로는 부족해 정용진 부회장도 나선 이마트 상생 스토리'],
             '인터파크': ['인터파크 작년 영업익 163억원 전년比 268% 증가'],
             '한진': ['한진 조현민 이대 약학관 방문 섬유화질환 연구센터 후원'],
             '한화갤러리아타임월드': ['한화갤러리아타임월드 지난해 면세점 철수로 순손실 482억원 적자전환'],
             '현대건설': ['현대건설 협력사 근로자에게 마스크 무상 지급'],
             '현대백화점': ['현대백화점 우한폐렴 우려에

In [196]:
def save_json_newsdata(dict_data:dict,url:str,save_dir='crawling_data'):
    save_time = time.strftime('%Y-%m-%d', time.localtime(time.time()))
    crawl_name = url.split("//")[-1].strip('/')
    file_name = f"{save_time}_{crawl_name}.json"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    save_path = os.path.join(save_dir,file_name)
    with open(save_path,"w") as json_file:
        json.dump(dict_data,json_file)

In [197]:

save_json_newsdata(company_dict,url=url)